In [ ]:
! mkdir ~/.kaggle


In [ ]:
!cp kaggle.json ~/.kaggle/


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download kapillondhe/american-sign-language

In [ ]:
!unzip american-sign-language.zip

In [ ]:
import numpy as np
import PIL
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
batch_size = 32
image_size = (400, 400)

In [ ]:
# load train and validation images
train_dataset = keras.utils.image_dataset_from_directory(
    'ASL_Dataset/Train',
    validation_split=0.25,
    subset='training',
    seed=121,
    image_size=image_size,
    batch_size=batch_size
)

validation_dataset = keras.utils.image_dataset_from_directory(
    'ASL_Dataset/Train',
    validation_split=0.25,
    subset='validation',
    seed=121,
    image_size=image_size,
    batch_size=batch_size
)

In [ ]:
# Listing classes
classes = train_dataset.class_names

In [ ]:
# #data augmentation
# AUTOTUNE = tf.data.AUTOTUNE

# train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
# validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(400,
                                  400,
                                  3)),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
  ]
)

In [ ]:
num_classes = len(classes)

model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

In [ ]:
epochs=5
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=epochs,
  callbacks=callbacks
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig("validation score.jpg")
plt.show()

In [ ]:
model.save('asl_model_2_layer.h5')

In [ ]:
# image_path = '/content/ASL_Dataset/Test/M/3001.jpg'
def predict_model(image_path):
  img = tf.keras.utils.load_img(
      image_path, target_size=(400, 400)
  )
  img_array = tf.keras.utils.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # Create a batch

  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])

  return score

In [ ]:
import os
main_path = 'ASL_Dataset/Test'
y_pred = []
y_label = []
for label in os.listdir(main_path):
  parent_path = os.path.join(main_path, label)
  for image in os.listdir(parent_path):
    image_path = os.path.join(parent_path, image)
    score = predict_model(image_path)
    y_pred.append(classes[np.argmax(score)])
    y_label.append(label)
    print(
      "Predicted Label: {} with a {:.2f} percent confidence, Actual Label: {}"
      .format(classes[np.argmax(score)], 100 * np.max(score), label)
    )